## 챕터 9: 파이썬스러운 객체

* repr(), bytes() 등 객체를 다른 방식으로 표현하는 내장 함수의 지원
* 클래스 메서드로 대안 생성자 구현
* format() 내장 함수와 str.format() 메서드에서 사용하는 포맷 언어 확장
* 읽기 전용 접근만 허용하는 속성 제공
* 집합 및 딕셔너리 키로 사용할 수 있도록 객체를 해시 가능하게 만들기
* _slots_를 이용해서 메모리 절약하기

### 객체 표현
* repr( ): 객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환한다.
* str( ): 객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환한다.

### 벡터 클래스의 부활
- Vector2d 클래스 사용

### 대안 생성자
- frombytes()
- array.array

### @classmethod와 @staticmethod

In [4]:
# @classmethod와 @staticmethod의 동작 비교

class Demo:
    @classmethod
    def klassmeth(*args): #klassmethod()는 모든 위치 인수를 보여준다
        return args
    @staticmethod
    def statmeth(*args):
        return args

In [5]:
Demo.klassmeth() #호출 방법에 무관하게 Demo.klassmeth()는 클래스를 첫번째 인수로 받는다.

(__main__.Demo,)

In [6]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [10]:
Demo.statmeth() #Demo.statmeth()는 단지 평범한 함수처럼 동작할 뿐이다.

()

### 포맷된 출력
format() 내장 함수와 str.format() 메서드는 실제 포맷 작업은 _format_(format_spec) 메서드에 위임한다. format_spec은 포맷 명시자로서, 다음 두 가지 방법 중 하나를 통해 지정한다.
* format(my_obj, format_spec)의 두번째 인수
* str.format()에 사용된 포맷 문자열 안에 {}로 구분한 대체 필드 안에서 콜론 뒤의 문자열

In [11]:
brl = 1/2.43
brl

0.4115226337448559

In [12]:
format(brl, '0.4f')

'0.4115'

In [13]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.41 USD'

각 클래스가 format_spec 인수를 자신이 원하는대로 해석해서 포맷 명시 간이 언어를 확장할 수 있다. 예를 들어 datetime 모듈의 클래스들은 자신의 _format_() 메서드에서 strftime() 함수와 동일한 포맷 코드를 사용한다.

In [14]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'15:43:27'

In [15]:
"It's now {:%I:%M %p}".format(now)

"It's now 03:43 PM"

In [16]:
# Vector2d 클래스 내부
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({}, {})'.format(*components) # 포맷된 문자열을 '(x, y)'형식으로 만든다.

In [17]:
# Vector2d.format().버전#2
def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('p'): # 포맷 명시자가 'p'로 끝나면 극좌표를 사용한다.
        fmt_spec = fmt_spec[:-1] #fmt_spec의 마지막에 있는 'p'를 떼어낸다.
        coords = (abs(self), self.angle()) #(크기, 각)으로 극좌표 튜플을 만든다.
        outer_fmt = '<{}, {}>' #꺾쇠괄호를 이용해서 바깥쪽 포맷을 구성한다.
    else: #그렇지 않으면 self의 x,y요소를 이용해서 직교좌표를 만든다. 
        coords = self #괄호를 이용해서 바깥쪽 포맷을 구성한다.
        outer_fmt = '({}, {})' # 괄호를 이용해서 바깥쪽 포맷을 구성한다.
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)

### 해시 가능한 Vector2d


In [ ]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.__x, self.__y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        return hash(self.x)^hash(self.y)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    @classmethod
    def frombytes(cls, octects):
        typecode = chr(octects[0])
        memv = memoryview(octects[1:]).cast(typecode)
        return cls(*memv)

### 파이썬에서의 비공개 속성과 보호된 속성

### __slots__클래스 속성으로 공간 절약하기
- 기본적으로 파이썬은 객체 속성을 각 객체 안의 _dict_라는 딕셔너리형 속성에 저장한다. 딕셔너리는 빠른 접근 속도를 제공하기 위해 내부에 해시 테이블을 유지하므로 메모리 사용량 부담이 상당히 크다. 만약 속성이 몇개 없는 수백만개의 객체를 다룬다면, _slots_클래스 속성을 이용해서 메모리 사용량을 엄청나게 줄일 수 있다. 이 속성은 파이썬 인터프리터가 객체 속성을 딕셔너리 대신 튜플에 저장하게 만든다.
- _slots_를 정의하려면, 이 이름의 클래스 속성을 생성하고 여기에 객체 속성 식별자들을 담은 문자열의 반복형에 할당한다. 불변형인 튜플을 사용하면 _slots_정의를 변경할 수 없음을 알려준다.

#### _slots_를 사용할 때 주의할 점
* 인터프리터는 상속된 _slots_속성을 무시하므로 각 클래스마다 _slots_속성을 다시 정의해야 한다.
* _dict_를 _slots_에 추가하지 않는 한 객체는 _slots_에 나열된 속성만 가질 수 있다.
* _weakref_를 _slots_에 추가하지 않으면 객체가 약한 참조의 대상이 될 수 없다.

### 클래스 속성 오버라이드
- Vector2d 클래스에는 typecode라는 클래스 속성이 있다. 이 속성은 _bytes_() 메서드에서 두 번 사용되는데, 우리는 단지 self.typecode로 그 값을 읽었다. Vector2d 객체가 그들 자신의 typecode 속성을 가지고 생성된 것이 아니므로, self.typecode는 기본적으로 Vector2d.typecode 클래스 속성을 가져온다.